In [1]:
# TEST: Imports
from seccom import Seccom
import datetime, secrets, binascii, os
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import ec, rsa
from cryptography.hazmat.primitives.kdf.concatkdf import ConcatKDFHMAC
from Cryptodome.Cipher import PKCS1_OAEP

In [2]:
# REQ: Peer Keys
# B Keys
ec_private_key_b = ec.generate_private_key(ec.SECP256R1(), default_backend())
ec_public_key_b = ec_private_key_b.public_key()
ec_public_key_b_hex = ec_public_key_b.public_bytes(encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()
# Generate a new RSA private key
rsa_private_key_b = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
# Define the subject of the certificate
subject = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "DE"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, ""),
    x509.NameAttribute(NameOID.LOCALITY_NAME, ""),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, ""),
    x509.NameAttribute(NameOID.COMMON_NAME, "0.0.0.0"),
])
# Create the certificate
cert_b = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(subject)
    .public_key(rsa_private_key_b.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.utcnow())
    .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
    .add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    )
    .add_extension(
        x509.SubjectAlternativeName([x509.DNSName("127.0.0.1")]), critical=False,
    )
    .sign(rsa_private_key_b, hashes.SHA256(), default_backend())
)
# Print the private key and certificate in PEM format
private_key_pem_b = rsa_private_key_b.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)
cert_pem_b = cert_b.public_bytes(encoding=serialization.Encoding.PEM)

In [3]:
# TEST: Initialization
seccom = Seccom()

In [4]:
# TEST: ECDHE
resultEcdhe = seccom.ecdh(ec_public_key_b_hex)

In [5]:
# TEST: RSA Exchange
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(resultEcdhe['sharedSecret'])),
                modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
encPKRSA = encryptor.update(cert_pem_b)
resultRsaE = seccom.rsa(encPKRSA.hex(), iv.hex(), resultEcdhe['sharedSecret'])

In [6]:
# TEST: Encrypt and Decrypt
inputPlaintext="abc123!)?"
resultCipherText = seccom.encrypt(inputPlaintext.encode('utf-8'), cert_pem_b, resultEcdhe['sharedSecret'])
seccom.rsa_private_key = rsa_private_key_b
resultDec_plaintext = seccom.decrypt(resultCipherText['ciphertext'], resultCipherText['iv'], resultEcdhe['sharedSecret'])

In [7]:
# RESULTS:
print('Output ECDHE Handling:')
print(resultEcdhe)
print('\nOutput RSA Handling:')
print(resultRsaE)
print('\nPlaintext Pre-Ciphering:')
print(inputPlaintext)
print('\nCiphertext:')
print(resultCipherText)
print('\nDecrypted Plaintext:')
print(resultDec_plaintext.decode('utf-8'))

Output ECDHE Handling:
{'sharedSecret': '26f330fc46a4232ba079752b94e880c70348d5e56391eca3e0ed091a710ae341', 'payload': {'digest': '903e7f34c9f1accb0e57694aaa031bdcd803b3bb2edd6975aca66609b2bd8ba7', 'key': '2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d466b77457759484b6f5a497a6a3043415159494b6f5a497a6a304441516344516741452b39766b6f50654234742f792f4f76334953774365514e4f36566e660a2f44723832446241587371746246695a63425978587a4e68685672765153786d6a3353594c4b4e36732f37535641535a316b39547764326671773d3d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a'}}

Output RSA Handling:
{'payload': {'iv': 'b7f719e9b51021de6027a4ebfdb776a4', 'ciphertext': '446c3681456207f605ebf07f92b6ee5c21f47b49732b59119c3620345ac51b006902055f993654b35751f2bbcff3819b39d8f7d2d0c56f849058623b1f49e27be4d5039f95305891c20eca787f27041377439caabf2fcc02b91af0c2837cf3f133c9705be1661f1b818bb894a10e3db89ce7bf860843f55c0155614bbd49ed953f52094759580120236111931c572b78032468a4cc91950ef71011fb549a6e58dea4ee41fa506e057ecd58

In [8]:
# REQ: Both 'Parties' for Handshake
sec1 = Seccom()
sec2 = Seccom()

In [9]:
# TEST: Initiate Handshake on Peer (ECDHE)
req11 = sec1.ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()
req12 = sec2.ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()

resp11 = sec2.ecdh(req11)
resp12 = sec1.ecdh(req12)

In [10]:
# TEST: Finalize Handshake (RSAE)
req21 = {}
req21['iv'] = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(resp11['sharedSecret'])),
                modes.CBC(req21['iv']), backend=default_backend())
encryptor = cipher.encryptor()
req21['encPKRSA'] = encryptor.update(sec1.cert_pem)
req22 = {}
req22['iv'] = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(resp11['sharedSecret'])),
                modes.CBC(req22['iv']), backend=default_backend())
encryptor = cipher.encryptor()
req22['encPKRSB'] = encryptor.update(sec2.cert_pem)

resp21 = sec2.rsa(req21['encPKRSA'].hex(), req21['iv'].hex(), resp12['sharedSecret'])
resp22 = sec1.rsa(req22['encPKRSB'].hex(), req22['iv'].hex(), resp11['sharedSecret'])

res21 = sec1.decrypt(resp21['payload']['ciphertext'], resp21['payload']['iv'], resp11['sharedSecret'])
res22 = sec2.decrypt(resp22['payload']['ciphertext'], resp22['payload']['iv'], resp11['sharedSecret'])

In [11]:
# RESULT: Handshake
print('Clear-text Key 1:')
print(sec1.cert.public_bytes(encoding=serialization.Encoding.PEM))
print('Clear-text Key 2:')
print(sec2.cert.public_bytes(encoding=serialization.Encoding.PEM))
print('Request Initial Key 1:')
print(req11)
print('Request Initial Key 2:')
print(req12)
print('Peer Response for First Request 1:')
print(resp11)
print('Peer Response for First Request 2:')
print(resp12)
print('Rsa Request 1:')
print(req21)
print('Rsa Request 2:')
print(req22)
print('Rsa Responese 1:')
print(resp21)
print('Rsa Responese 2:')
print(resp22)
print('Rsa Responese Deciphered 1:')
print(res21)
print('Rsa Responese Deciphered 2:')
print(res22)
print('Val 1')
print(str(sec1.cert.public_bytes(encoding=serialization.Encoding.PEM) == res22))
print('Val 2')
print(str(sec2.cert.public_bytes(encoding=serialization.Encoding.PEM) == res21))

Clear-text Key 1:
b'-----BEGIN CERTIFICATE-----\nMIIDNDCCAhygAwIBAgIUFp56SFXex05frCyPXtZw81xzdCEwDQYJKoZIhvcNAQEL\nBQAwQDELMAkGA1UEBhMCREUxCTAHBgNVBAgMADEJMAcGA1UEBwwAMQkwBwYDVQQK\nDAAxEDAOBgNVBAMMBzAuMC4wLjAwHhcNMjMwNTA4MTIwMTA4WhcNMjQwNTA3MTIw\nMTA4WjBAMQswCQYDVQQGEwJERTEJMAcGA1UECAwAMQkwBwYDVQQHDAAxCTAHBgNV\nBAoMADEQMA4GA1UEAwwHMC4wLjAuMDCCASIwDQYJKoZIhvcNAQEBBQADggEPADCC\nAQoCggEBALMNqU5qOer69rTQS2aa89fSLNY32DJ8vLeHeQ1ROpBmdtM9/NHt312l\n+t4kYSNevqZdrOpdngOSsABuntePQODboCYt42+G1DXaC/eD+QVOdIf1PopOzSp6\n22YCYryTa/DLl6fRnEm33LR8qsDEJtTbKVLDo8R0D53bB19IXmGLV805BFM4cjcN\nYi/WWi/piExn/umQUJl/XB/qw3/v2EIcuTMJM27NgPjhTQ9MdCKwkJMOTspIkTFM\npeXwLS6lftOmzsW8/P80iLBVERcLFCQiD9KYL2wX/lHEobXLbw8DT2MJveABZQRZ\nHH0bYSnspqnSbmvmTpxy7CBJULdaaoUCAwEAAaMmMCQwDAYDVR0TAQH/BAIwADAU\nBgNVHREEDTALggkxMjcuMC4wLjEwDQYJKoZIhvcNAQELBQADggEBAKA41PX8gDek\nq/ngANTZo1cwuurIWQkggdM2eust4gBkli3chJrwftllSSEEpGYgfJNrPwxCNwWq\nO4lvfFNUQklJRb6GOCeX/S0LTk2NTaZEljSK9297hbeiEvIvZ2YaNg7YpKw9X9Bq\nUe2q4/uU3O7HRXtZj8l7vrDFqWs

In [12]:
#IMPORTS, CONSTANTS, TEST-FUNCTION:
import requests, json, string, random

apiTokenValue = 'LlLvXURqNHrp75iyz9Riwen4DJ2QcfDK78sCw4tgjgPF4gsFcIJuPbQfrUMm'
userTokenValue = 'hmLHka03PSxsE73bKnD4xNXlPmy6UF7oiVNHANEvxAZOCsXz0J9DX82JjyBR'
creatorTokenValue = 'q98lD6P6s4jQWO8yMUZalvQZqZ2pQq3QHZLnwQKhXPhocrVYIuWTEYBOJta9'

def testRequest(methode: str, url: str, headers: dict, cookies: dict or None, body: dict) -> dict or Exception:
    resp = {}
    if methode == 'POST':
        try:
            response = requests.post(url, headers=headers, cookies=cookies, data=json.dumps(body), verify=False)
            if response.status_code == 200:
                resp['payload']: dict = json.loads(response.text)
                resp['csrfToken'] = response.cookies['X-CSRFToken']
                resp['sessionId'] = response.cookies['sessionId']
            else:
                raise Exception('Request failed with status code ' + str(response.status_code))
        except:
            raise Exception('Request failed without status code ')
    elif methode == 'GET':
        try:
            response = requests.get(url, headers=headers, cookies=cookies, data=json.dumps(body), verify=False)
            if response.status_code == 200:
                resp['payload']: dict = json.loads(response.text)
                resp['csrfToken'] = response.cookies['X-CSRFToken']
                resp['sessionId'] = response.cookies['sessionId']
            else:
                raise Exception('Request failed with status code ' + str(response.status_code))
        except:
            raise Exception('Request failed without status code ')

    return resp

def createId(length: int) -> str:
    letters     = string.ascii_lowercase
    result_str  = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [13]:
# TEST: Ecdhe Request
sec3 = Seccom()
#sec4 = Seccom()

ecPkHex = sec3.ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()

resp0 = {}
try:
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'ecdhPublic': ecPkHex}
    resp0 = testRequest('POST', 'https://0.0.0.0:22432/handshake', headers, None, body)
except:
    raise Exception('Request failed with/without status code ')

localEcdhe = sec3.ecdh(str(resp0['payload']['key']))
sharedSecret = localEcdhe['sharedSecret']
valEcdhe = localEcdhe['payload']['digest'] == resp0['payload']['digest']
print(resp0)
print(valEcdhe)

{'payload': {'digest': '60bfe915ebe2173619c4e3f5d78f8c965fa544da694e30deeb5fac5218c0167e', 'key': '2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d466b77457759484b6f5a497a6a3043415159494b6f5a497a6a304441516344516741454237437338646d793759506a32726e474177632b766d4e65366f2f360a59386c4b75692b662b6c722b44484776526c6f5a4d6957394951653761315a37362f7970304e6531583837446d764d447877395a4450517736673d3d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a'}, 'csrfToken': '8f64343a59d7d1b219e70979a914a4a57d0154daIjQ1ZDg2NGM0MmU2OThlMGE4ZjVmZDg0MzM5ZGU3NTIzY2ZhOTEzYmIi.ZFjkhg.z45C-BlrQ_15tFAl2z7xbGID44k', 'sessionId': '733676a3-bdb0-48c3-ad26-8f856ca313aa'}
True


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
# TEST: Rsae Request
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(sharedSecret)),
                modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
encSymRsaPk = encryptor.update(sec3.cert_pem)

resp1 = {}
try:
    cookies = {'sessionId': resp0['sessionId'], 'X-CSRFToken': resp0['csrfToken']}
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'iv': iv.hex(), 'key': encSymRsaPk.hex()}
    resp1 = testRequest('GET', 'https://0.0.0.0:22432/handshake', headers, cookies, body)
except:
    raise Exception('Request failed with/without status code ')


print(resp1)
peer_rsa_crt = sec3.decrypt(resp1['payload']['ciphertext'], resp1['payload']['iv'], sharedSecret)
print(str(peer_rsa_crt))
print(str(peer_rsa_crt.hex()))


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'iv': 'c61fbfa96d3d64c33f8dc01c42312175', 'ciphertext': '505065e1c5015d7a7dbf75c5befdd0bf58361255e535e5ba7c7677b8b85caa279e07d25445343de39564693a865c27b2cc5cebe2f6bf25794c4f53a9013a85cff136c113c3010a0f5065e4289a08ebb5f2ef55cef86b549bdd832210dbf1660ee216b54dd1046271087941c8196d1de9b299f703fc6da68dde675a5fc7549abb9ae2397365f6078cb7c416e9d0388bfc275812d7a9b4cc2e24fcbe62a748ee08d38db7c80fb8bfa11ee087862c49515e8addbfb7e8bbf496f260eac9c6639b3313725d0ce6b0ef8a34c980afbdaee7486290f8b7fe32831ec850f982017a6aa25382c6f9dd27c3f07f285cbbb4c97484b86f5a847a9a736b635473f1dd8f846dad16e821c471f8b6038c62af9949a2989031ca9b2331a9f16ee3a921191297fa6cb0ef3d8d7a4e2fa6d358215139ed6f4af1e87f416db945ec92ddea2367892ac881cfbcc0ebd8f5d5ab24088cf5322b327384d36775c98418a95ec5eebe42f94b09a3a3d5afa119336b8134313ae3d3ba1ca7c03e02cd7fd931ce52b15ac2faebdafbc80f57aff596d615c6a1092c0be9ac4e33a0cd72a09a47f92ddaff8ebe86140ee04be5a3f3ef411d8db8d634b8972b8aaf8d8bda37fec35d685eb9f39f55a93b903684c86cbf288fabfe4085d813

In [15]:
# TEST: Distribution Request
resp2 = {}
try:
    channelName = createId(5)
    channelName_hex_string = hex(int.from_bytes(channelName.encode(), 'big'))
    peer_hex_string = peer_rsa_crt.hex()
    encRes = sec3.encrypt(bytes.fromhex(channelName_hex_string[2:]), bytes.fromhex(peer_hex_string), sharedSecret)
    encIv = encRes['iv']
    encText = encRes['ciphertext']
    cookies = {
        'sessionId': resp1['sessionId'],
        'X-CSRFToken': resp1['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'userToken': userTokenValue, 'iv': encIv, 'channelName': encText}
    resp2 = testRequest('POST', 'https://0.0.0.0:22432/distribution', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp2)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'REQUEST': 'SUCCESSFULL'}, 'csrfToken': '9c6a93b2098e8cc329be8b19a60bc859d79cbb86.eJwdydtugyAYAOB36QM0_CjW3jqph0Q6GcrqzQLFlQraxeygPv2WfbffDmODDwYHJjYIaYQA94BjjcwxPPQhVuoYvkf4CMVgRzbUG0uZZcI7huvlMhTLWTZBlbrgLCmpBLcso_B3P5VoUJedSuXpIh2fKkdyIRKm22YVwB-aQs4FV8p3n_9PIRKiVRyaTXhe9t6tcrTPLW0jKaFUL0Tx8WNW2fWrH2qoV3jSeclfNwqGdolOy4gF9s6Rpyzlc3Hbd6fB2dv-cXHTm7TNZH1NSEySbz_3U3m9Y7f7BXY2UcU.ZFjkiw.WEyWyWnA8_CnclcZ9UzulwCRXuA', 'sessionId': '733676a3-bdb0-48c3-ad26-8f856ca313aa'}


In [16]:
# TEST: API Ca Creation Request
resp3 = {}
try:
    cookies = {
        'sessionId': resp2['sessionId'],
        'X-CSRFToken': resp2['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp3 = testRequest('POST', 'https://0.0.0.0:22432/cert/api/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp3)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'REQUEST': 'SUCCESSFULL'}, 'csrfToken': '94c92946e29232e30e4449981c42aa7d95dd0a04.eJwNyNu2QkAAANB_6b2WZgrziJJKch-8WGYYErmUNL7-nP24V4iKGYIECEguZEohQKSQyRZlokCovEe5hCghsrgpLnPO889UciVW7ioUnZuQajWG9dBXgiOe6Dgsjqk3zpj9Cl7aX4AZYezQ3g-HOTYudXzOYkdBO7Wu6fyzbacKpy7-PtlOu4Wlu5DgJExJQs9SNX2bW-Wb2E_UhoylqbW9atJ3l6rQ3oddcClyb4r6cwXY9Znzs3ttAYjDWV1nznunh3IPXymy__8Rujzzo6pAH_FDR982sVBfMz0wBf8nYwsDhkcDDF0Ia6-gDbPm4ZR3zZ3vSh29RbfnS0OhQXKRqSD_rmlk-K3kW4mxtbwj98iSwgC2KJaYpEYxCGiwSfT6-Zg3-Mgxxy9FTrUXbWiCgmVqZs2NJmX1Bzrrfnk.ZFjkjg.pBAFn5az8DLodDNE_y-EZgeV43o', 'sessionId': '733676a3-bdb0-48c3-ad26-8f856ca313aa'}


In [17]:
# TEST: API Ca Request
resp4 = {}
try:
    cookies = {
        'sessionId': resp3['sessionId'],
        'X-CSRFToken': resp3['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp4 = testRequest('GET', 'https://0.0.0.0:22432/cert/api/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp4)
print(sec3.decrypt(resp4['payload']['caCrt']['ciphertext'], resp4['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp4['payload']['caKey']['ciphertext'], resp4['payload']['caKey']['iv'], sharedSecret))


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': '32d09f97c0a535c8aa25e420b7ab92cf', 'ciphertext': 'e8935e8f41dc4f3a0f5fe6ef1b11503757848ce2a5e211d2a6ef92db49e06f754739ad5574850e6b31798f8cfe22d79b061f11764f61d871e78454c3d9750ffaf0e7dfeb5f6af15cf9bbcdcd24520149e453a3f1015d99cb39d1c9d3148a1b17733377c40bebab8cfb49a47bbf8ab164b6cd3eff9773799eb2b5433fd882aba89705096db3ed6fb458ff76616de39d494f366d99b701e701ec0bbbb002d404a8f55efc515ec2c98798771f3e24d1f7f793f0c5787a816ddd024711e7b888aaa0f880149d50a0c27d84191cf54b42864b6d7b5e031c9adf981ed5420f305613a6a458f093ecb9c386505c85e5e0e63c09e1188fa7485eac634248ec05edaca0d663762152a1d593cc57528a55c2567658963450d49de767c46600971996471b8bef726151e3000f3610af075635dbc344cad2fbb62c32b651098f29d6f2c6631e41375e96445939a882e2d008954e51e99e934d55e70577b53a06884d16ae4145278be1835c7d90377b0c6e3ced8c2bd5848fc59e0f793868f85fe3eabb1658099dc66608171e408a63d88c83430ab25e2f15c5ab35bcd5cb3e1beec670ed36ac148363714597c2abc6f30e96f6394f09caa4579b3cc10d2b4aa3a73fede1e62e4cda77d56467e710b8c8b195

In [18]:
# TEST: API Broker Crt Creation Request
resp5 = {}
try:
    cookies = {
        'sessionId': resp4['sessionId'],
        'X-CSRFToken': resp4['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp5 = testRequest('GET', 'https://0.0.0.0:22432/cert/api/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp5)
print(sec3.decrypt(resp5['payload']['caCrt']['ciphertext'], resp5['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp5['payload']['caKey']['ciphertext'], resp5['payload']['caKey']['iv'], sharedSecret))

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': '0d79cf5a7fa19fc3525b704ec5977424', 'ciphertext': 'd9dcccbbfa1360894db13200124a4e1f16f99a918a3d121a6011af712dca36315368c800729677502ddb8ccc35db20bc46898503e52b1a9db93fce4da64add41dea9d8ac08683b7a1bf74d129a7c062ca9ae226c4b9ae1135ad41df387a37216d25b92516cc796e4425c15f888d374ab05de200de600bf95e4a916a721b3a5fa24f2565c96cb072dd273061d9b60b47acb6f8a11f6e6159bc8c6d27832b9b0537abf6348d5d448d8db437dedb154e452ba818b44c0ee3615cac1ab9971335d46b71b10c87d5c81bf76fcfd7f4b70795b5120b61ed4423c5bdbca78e6437e91289a140de85072fd8f4365e326369da9d1c8c7e11fb39806c8292aedc1c523a03c9fa85e57046c2368656f9776af69510731de6a0a2ebbfcf894f4f691d6a636505feb61bd227f65c5918b9622b93e2910a4c204bc40735972b04fb765e5ec303fbbb1f46bc6ac70a2f4a8ad7f401502cb79bbb771db3002e6a67a9079ba3fcdd5c4cfd76d11f8eccdbd0211dd4877c93b0b9fafed6f5b8c26300980286a3eed9885ba0d67ba0564463a31cf9d09dcc97f405232730fe1b438b0ecb8497f75a388b66a81068f1b7fd7187850c623587f85ffc9540b0c96fdc47b2de8c920774ce1ca079276e9463ed38fae4ba1

In [19]:
# TEST: API Broker Crt Creation Request
resp6 = {}
try:
    cookies = {
        'sessionId': resp5['sessionId'],
        'X-CSRFToken': resp5['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp6 = testRequest('POST', 'https://0.0.0.0:22432/cert/api/broker', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp6)
print(sec3.decrypt(resp6['payload']['caCrt']['ciphertext'], resp6['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp6['payload']['caKey']['ciphertext'], resp6['payload']['caKey']['iv'], sharedSecret))

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': 'b9fcd43b9bcf61a74101b426c8aa730d', 'ciphertext': 'd50d8980e438d0c2b7abfed82db128907408915cf17d96657511575d0c65a4ab0c27bfa467bf8d233b48eade310dcfa4dadc318e802b8edab5ac356b418a9440d076dadd2b565642d921d225a6f51e96a34e3bb172db8aa6e04d0f445d2862818ae9c42748a3daae54ca152bed0c520fbcebd2731ccb0d6414c3deba2c07be18ef914db8f13a3464db6925d631336667eb7b0f8190a32dfef80d1c56338139bbfa8377286fc7a23ab7f95a04b7e0122a985c464b834a4aef29181f435bc25409b433fd34ffae03aa7140d00725efc085844dcf6ec96b405e12dcf6371f98497d50c1485a6f708dee7627ce63747b08f74c317d299239e88e70b7bf1fce230f5e743898a6583b26cd9ead251f83a8800e5ca1b18d5c44449f176576d208a1d1ebfd228222965bc7403bdb648fbeb401614756d5edd79b422d5289c954f927ae5a97825d638df62b442b6d31884796a3fe32ab39886fad19ae0c8c92c1bded80b1a5d0756ea55d4ee17d012fef42cff57887fb6ad905994995a1d42a634ddabf3f09571a70f0f0741b008696bacd540e1e0eedb00b6617d1c750d7015ddd29483958ddad595d938ab879affbdfe2d1b14ecfc469707f7fd82925b74f01638d9541aae4b6d6b2fe704f07fb2266

In [20]:
# TEST: API Client Crt Creation Request
resp7 = {}
try:
    cookies = {
        'sessionId': resp6['sessionId'],
        'X-CSRFToken': resp6['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp7 = testRequest('POST', 'https://0.0.0.0:22432/cert/api/client', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp7)
print(sec3.decrypt(resp7['payload']['caCrt']['ciphertext'], resp7['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp7['payload']['caKey']['ciphertext'], resp7['payload']['caKey']['iv'], sharedSecret))

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': '963947bc70d34b226568c0f82ecb6ca2', 'ciphertext': '68b7bd92fbdc5740f43b0f21ad4ba96bc460a9a648dc658fbbac23a4151a97915983845a620d014fa2df6754e423b4a6c4cb91f9caa6d5c82f8553c3e8cf43765fc511328f57c0cfaa1c4a8df6663c3dfe41f8f01a225ea462f7bd4abe084483d7a17e21b9cf50faeca94781ac3e9299336c5d110afcf1502ca7e4818db0e13507431ada9fbfc7a9c5521cca70c5b43053751ef5dff11b294e3c9b5e9426f62b38928d9949df2df7b0475a19971b5160e3424117070de5114f2ca47a6a3fe4a7d00258f79407bba4e12f1772acbee5da8d43d1145ab362ac09949ab1b43a83a17177e4af59b74a4b683d3f70b0ca5f0e5dc9f7b22fa438873487bfb682f194f2a06fcaf6494c87fd81d42b5a87b7c6e9f0f9a636cb24c9c1f3bfec42db272e4db27f07bd8316a6dcf148ed4efcd27f3d72fb9040a61ba0c17479975e83d60bd6ca56880d57e5237415640ed2fadbae51a225ace75319bcc0f2b151126031d4287cc4737cf6921ee74cff228606507ec24e15ff282874f8a137f28b0b42148b212d1dc6684e443f92866f4ebb0af3985082aed88c47add9727062c2e4585e9b7a5fbc3b1fc11c0a6c5b4b8ed9132e1ac40b1873686365d53f7a283cf05eda8ea0fe369ad1ad6bcd771bfae4b4

In [21]:
# TEST: CAM Ca Creation Request
resp8 = {}
try:
    cookies = {
        'sessionId': resp7['sessionId'],
        'X-CSRFToken': resp7['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp8 = testRequest('POST', 'https://0.0.0.0:22432/cert/cam/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp8)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'REQUEST': 'SUCCESSFULL'}, 'csrfToken': '3aba7d570ada74f1c9acd0896748fa6aca98fd7f.eJwNzke2qkgAANC9_DnvUGSGJaDknCd1iqRkEERh9d13BfefADgsUIAjKxFUmMZFwXKAFWiWoUAJOJ4RcVmDgvqr9a9NKmUw9xBCS0K8Te2ad72CeyD1bai9El3r-_tTMlCDonuKToX9mbYQLN7g4Yz8XXYevmEOqfZhhM_Emp-qcpMgAY6ZII7WFvXlFt2ZNhti_Z6Je1bvXu2t16uzPygzOMzrpez1IZ8wz1S7YqTUjrMA9x39HJMzCZBvZl0TPb7J4CcJZ7elNRgGMOo5QpXoEDQdk6L8USHu-e17QhQcXRSWTofnRVbXybV4wTuGEOHHdIDPcNYnz_cEYZVCDZPlrVVIdtMn37IvRz6daBHdXSaoGSvt5ZGhlNadtU1CnI5P6kZlkTyUohTQ0z4YBaX7Nx-z4nUotXkgzRW69Js3Gu3mpzcR6rt6NMp1aFPSJPqd0UzXUY782JcR5reiu3sjzdQLYhUqyzfS7U-5iSu34drXh0f4o79bPxDX0-KfOvv_E34vULw4JmRGV6HpzZ49RmqD12MIEvXFtiFLpel773Suz05tG2rVeWYO03tn2mI4nYpIL807rX4WcUF_PVr2qDP_jLdpzXuoBDJbZZHe6AlppgIoJdJzA9cMoUIKuhL8Ftm7R-D7xX5RRh5SfMp9Gg9bBV__AIVmhnQobGwWU-MlSaFHl6y5Tq0unN-RLN24uzxnuA5fGg2Z1S1jSNnSEAD6UFfUDBUK2Vyfq15orRD8JkdUMznjbnubmLRBOLugDdiq-qs0FI1yqiU-95HyMqGWqd3qMQlBCv0fR-a4KxlSxFXBGeEr6kvi7ZEr9ONjW8iCYAkFpHNhLMWb_rRTY0pDPKoUQjlHq_gST9jIRh0td0mamUq2V8vmkpEFVCbqvpKek

In [22]:
# TEST: CAM Ca Request
resp9 = {}
try:
    cookies = {
        'sessionId': resp8['sessionId'],
        'X-CSRFToken': resp8['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp9 = testRequest('GET', 'https://0.0.0.0:22432/cert/cam/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp9)
print(sec3.decrypt(resp9['payload']['caCrt']['ciphertext'], resp9['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp9['payload']['caKey']['ciphertext'], resp9['payload']['caKey']['iv'], sharedSecret))

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': '32d64b9584dee8f79241fddcaa9256b2', 'ciphertext': '8ad102bf5cac112966b81069b9ec650df581151d3f275432071cc68910da24c7f3e182e3588db169cf21fb5089c2585fa1b0fdaa913a89ae393576a4f880499f3d4e5f5a27d73b454cd05fd833274a8da81138114579de663677d54c74f9a3ef0fd7d0ee40b560c1e52eb9bbfcb9a02fbf54d43e0bbd2903e6426fad7727811f57f32d1e03f5850c715aadc253bef02adc57cc6e47fa0585d6477dbf723fbc6ceafe006bfded0a26bff80f62ec3b2df8b705a0385c599de9dd4bad7fe6dd33126e881676141648e1e3249c1342bb90b950e2cb9767a88a9900e9c6ab537c6d2d19a762be4bce544bc246598673517fdff8d2e119e8dc3ca27f56a5b19b4f98a202b6721e2307291bb6931bba82a4dae798f762f49f2b51529266c702e74a70ad4ce1dec9e46590cda39d351c78628c7e3dd6067025ebda03a984f6d4ab0567700668cbf62243b77b5d0501f84093f77a93e63ced3d3e01beb8083e856b0c644279c1f8ab07950b1d7e8273fd11ecdfdc80554a51033731c5efc17e38743470312b0bf3d94fe0adb22380d16baae7d6fdd383800abb9945d0a3a294830c91fe69b55886561bbc1a1e1285c5d87598cb6128704f55323f3dd02dd137ca0bb755ef2eeee8b90a8594f7d91fe8d4

In [23]:
# TEST: CAM Broker Crt Creation Request
resp10 = {}
try:
    cookies = {
        'sessionId': resp9['sessionId'],
        'X-CSRFToken': resp9['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp10 = testRequest('POST', 'https://0.0.0.0:22432/cert/cam/broker', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp10)
print(sec3.decrypt(resp10['payload']['caCrt']['ciphertext'], resp10['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp10['payload']['caKey']['ciphertext'], resp10['payload']['caKey']['iv'], sharedSecret))

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': 'e69f913e085e8d8d1634257a8580bba7', 'ciphertext': 'b725437e10c7d9accdbd43913b99cdf29d35cb7ebb3850da10081d8166c2fc3ad78ff141cfc74ea3538eff96c10bf16a92f8548d49d470f43e7e5129ad22df58679782bf24623a93ee891dd8175231bc4a6e05ff09d83d59db4b9f3805ab6e55e53429e3444ac5ce2233f91aa0f55e6667864db47df967e89b489499b2c402acd8ae645f0253b37d7bcfd15f80fb8a839654a0b0310ee0a71d64bcf0cb06954620a9d729721462162f7dd02faa19e9f4e3db57cc9d4f74bdc276cda5e1d13c0082eacaf87e297a48120d36a9e3b58894f15b3d21178b2fdec46d32c96eb2751dd7e91d23f90de8dbe795807d16a82cca04fe1f1b0242c06afa46fc7a106e00bb54b09a066d11cefec082ce10a2c6b566526256a99618149a45e85aba6d712ba13f851716c00630bd1be8cf53fef13f3adfbe642547f7c2f45080e44e08e087da6e1a2aa1bc136890b63e35ce3cc95f52167f0ed58f09ed11d14982a771c71859f1acba4ab10fd3c47c8ed843528379f81c1cdf4e3c65e813d2a3491949577c0c1938374ad8080cc509a1215114bc114b6a7ccd97003da8daf0bacbad19f72cb689e1c9333f2317acf813ac7dbe471fe8185b9baf4c5b85a49f532c188ae4239f48938b808d43464f7d151002

In [24]:
# TEST: CAM Broker Crt Creation Request
resp11 = {}
try:
    cookies = {
        'sessionId': resp10['sessionId'],
        'X-CSRFToken': resp10['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp11 = testRequest('POST', 'https://0.0.0.0:22432/als', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp11)
#print(sec3.decrypt(resp11['payload']['caCrt']['ciphertext'], resp11['payload']['caCrt']['iv'], sharedSecret))
#print(sec3.decrypt(resp11['payload']['caKey']['ciphertext'], resp11['payload']['caKey']['iv'], sharedSecret))

{'payload': {'REQUEST': 'SUCCESSFULL'}, 'csrfToken': '8738e98530fb854b632efa9f459c960a0628ed99.eJwN0kUSo0AAAMC_7J0tLMxwxDW4XygsuIZgr9_9QR_6Dw1fdAEoHBJZSVegogqCRgFdvSBJwldWvMicKgvs9bdSTwNFfXdGGcY2JPCo5DIbwiDtOit7sQ9al5xkME6ITFv46KBuzdqidNdCotpsHLRDMbhJMo-d8A6xYBh2hUGwg_PRCsAFOOPvjdY-6BGcITdb2kfnDSd22hVCd0ex1JNMGZE0Fole2aPF3ikj6KcpzSLX2pvhu29WVFm5bDCwrKssrqtkNMH9kH3BhKvKfWnQm8fkZyVQzm9wuO24YuJGNrTUOtS97DJhYXEjJ6DSrnrnAs2px37gQuRLe63RYUqCv364Oic-BTVsji36jCNVFDBf6cpz8HX0JbYf7VeIv0bV85I4KmE5PV9HoExlkgTdEsRzXRXonAUOr-OCPcq0U5DT8QS3g8eF7MEa89Ikr1_cvJyoJ4zZ7BO-ad4zWjNOo491ozewzeIEVeGieLIX3Hh1nvCULlQ1xyQo9xz2FLe-liPm02szfM3pQ3T6NsNoAILyvq0j7FPWSNqCDTJzPy6ncM_jKv0XWsGip2vTJXL08oGJAAF0pb9EpsmEZupMZZyWromfFaFHfsOOC_rZNGt9XpzxfdWHPp8g1jpI86rfdZZITlSAsjD8BBxPAKOtBGbXnApdBUCTHEsRQ7hcTZ7GprPHTY81Pfr9pc8Aca53mEa_WFdHFOk5QsmYVjZdy2SJ5bTUABclcUtVgeKrlpkbSTcWz4cfP_m6WPoSXbHH64VCPXvma2-0Cii9yiVn_nTZhv4WeEt-1RNugTjJhS84V5Ild8mwX95B1eQasd28sEZdyidADG_QqcMN2Wb0CTEKDwW2ob61eYTpaVItgPOUlgbSt1jDxBYVV85qCsFk8eWeW

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [25]:
resp12 = {}
try:
    cookies = {
        'sessionId': resp11['sessionId'],
        'X-CSRFToken': resp11['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp12 = testRequest('GET', 'https://0.0.0.0:22432/als', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp12)

{'payload': {'REQUEST': 'SUCCESSFULL'}, 'csrfToken': 'ca25b48bdfde131ff48b1251ab72f910f1cd4d23.eJwN0Ueyq0gARNG99Fw_8KKGeCecKOyEEBROeI9Yfb8V5I2T_7FPki0AS5NYmbE0lTEkUZQfUFI0yAGDfTCGYAsEwL9CPy2s9b0R4zjOFNKnjm0v8zovMaSiX7XuWlItIAXuO2XEEy8RJ40qrydeUFRjNQvvSkaHx-tnh8LDbHKjOlaQIQ-ellxGSFHymNAdYaRFLTsbcbOb7bm64NPRV2tKjAblg2IMfr_R3CALCeM2rerrLZpZrZvjA7Ecc8V8TdS-clap8xZWTrZt5_ii-MHwSq5BlBFtOYgeIog6EBBWXLhumUoiYfLYFcTHkcmm_TJGPt3eKyKmo94JEAZyT2eicC7Gui7z1lqmClqV5GvjMMpwcHu2bJNf4N49OHeboOJdV6wF-vbmgaeo13EkfXVG9JZhGbiVmC7yWbn7-1Uw5Dv2Z-EgGcpu8gcPg_XbkMxXsAJZ5EvmL4NVI0zP4vIZFNqBZQFLa5TRS7QTAHWUrq6tIJLW9x29o3HgfHt52IKTz5hPi5B1PbJiC_lpSh8WaO2Cp_mh_0adupMnbz8-iLrDr2WPFMDHuzjvQpOCQmmKlxaRao0Ph-C_Ohe_fu4IrpuQX8Wja5weI9a7NMnJJSFjrdbIaa_fMWPfp-yEnjULItRv58cMuZl-jQOLwlVpJuKAevTC2FHROU7GpgxI09pLyT6ZSfILl7Ev5Y-slqttC-mSWMoGoum-ShSqzsRW-HfSWGYpUZJosPO49SuyPH85nGFvvJJFAzch3heg-ofhPuscJk9lWhwVxizulAuYcpbLPzTZS580lJEq2y3trn0cfBP0IzydzqDP8XmX-1tQFcbSTW3mwTy8qbx00p4JndGLMlVlqEBwoqP_EA9MaBrwa4IhHWBSH1RYSNsDf1t7Bb967

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
